In [134]:
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
import scipy.sparse
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

########################### DLEE IMPORTS ##################################

# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

In [2]:
# ##Download the pre-processed tweets from dropbox

# access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
# dbx = dropbox.Dropbox(access_token)
# metadata, res = dbx.files_download(path="/Inputdata2.txt")


# access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
# dbx2 = dropbox.Dropbox(access_token2)

# #Write the combined file into the drive
# with open("InputX.txt", "wb") as f:
#     metadata, res = dbx.files_download(path="/Inputdata2.txt")
#     metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
#     f.write(res.content)
#     f.write(res2.content)

In [112]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [113]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
#             if count == 500: break  # initially limit for time sake
    

In [192]:
#Read from the vocab file pre prepared
allword=set()
with open('vocab1.csv', 'r') as f:
    for line in f:
        allword.add(line.strip())

In [193]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))

Vocabulary size: 5,855


In [136]:
def cooccurrence_matrix(token_ids, V, K=1):
    # We'll use this as an "accumulator" matrix
    C = scipy.sparse.csc_matrix((V,V), dtype=np.float32)
    for k in range(1, K+1):

       
        i = token_ids[:-k]  # current word
        j = token_ids[k:]   # k words ahead
 
        data = (np.ones_like(i), (i,j))  # values, indices
  
        Ck_plus = scipy.sparse.csc_matrix(data, shape=C.shape, dtype=np.float32)
        Ck_minus = Ck_plus.T  # Consider k words behind
        C += Ck_plus + Ck_minus
    


    return C

In [117]:
def PPMI(C):
    """Tranform a counts matrix to PPMI.
    
    Args:
      C: scipy.sparse.csc_matrix of counts C_ij
    
    Returns:
      (scipy.sparse.csc_matrix) PPMI(C) as defined above
    """
    Z = float(C.sum())  # total counts
    # sum each column (along rows)
    Zc = np.array(C.sum(axis=0), dtype=np.float64).flatten()
    # sum each row (along columns)
    Zr = np.array(C.sum(axis=1), dtype=np.float64).flatten()
    
    # Get indices of relevant elements
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()
    
    ##
    # PMI equation
    pmi = np.log(Cij * Z / (Zr[ii] * Zc[jj]))
    ##
    # Truncate to positive only
    ppmi = np.maximum(0, pmi)  # take positive only
    
    # Re-format as sparse matrix
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape,
                                  dtype=np.float64)
    ret.eliminate_zeros()  # remove zeros
    return ret

In [118]:
from sklearn.decomposition import TruncatedSVD
def SVD(X, d=100):
    """Returns word vectors from SVD.
    
    Args:
      X: m x n matrix
      d: word vector dimension
      
    Returns:
      Wv : m x d matrix, each row is a word vector.
    """
    transformer = TruncatedSVD(n_components=d, random_state=1)
    Wv = transformer.fit_transform(X)
    # Normalize to unit length
    Wv = Wv / np.linalg.norm(Wv, axis=1).reshape([-1,1])
    return Wv, transformer.explained_variance_

In [174]:
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [194]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [195]:
#Set of all the stocks of interest
#trainstock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']

trainstock =['RiteAid']

In [196]:
##Converting the string of data into dictionary


for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in trainstock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
            
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [77]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [11]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Convert revenue into the y_label of 0 for negative growth and 1 for positive growth
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [13]:
y_keys = list(Outputy.keys())
print(y_keys)

['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty', "Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']


In [14]:
print(Outputy[y_keys[0]])

{'period': ['15Q1', '15Q2', '15Q3', '15Q4', '16Q1', '16Q2', '16Q3', '16Q4', '17Q1', '17Q2', '17Q3', '17Q4', '18Q1', '18Q2', '18Q3'], 'revenuechange': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [15]:
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [197]:
train_x_init = []
train_n_init = []
train_y_init = []
dev_x_init = []
dev_n_init = []
dev_y_init = []
test_x_init = []
test_n_init = []
test_y_init = []


In [198]:
trainquarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

for i in trainstock:
    for j in range(len(trainquarter)):
        try:
                train_x_init.append(InputX [i][trainquarter[j]])
                train_n_init.append(InputN [i][trainquarter[j]])
                length=len(InputN [i][trainquarter[j]])           
                train_y_init.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass



In [199]:
#teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
teststock =['RiteAid']
testquarter=['18Q2','18Q3']

for i in teststock:
    for j in range(len(testquarter)):
        try:
            if testquarter[j]=='18Q3':
                test_x_init.append(InputX [i][testquarter[j]])
                test_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                test_y_init.append([Outputy [i]['revenuechange'][j]]*length)   
            
            elif testquarter[j]=='18Q2':
                dev_x_init.append(InputX [i][testquarter[j]])
                dev_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                dev_y_init.append([Outputy [i]['revenuechange'][j]]*length)     
               
      
        except:
            pass

In [200]:

train_x_init2 = []
train_n_init2 = []
train_y_init2 = []
dev_x_init2 = []
dev_n_init2 = []
dev_y_init2 = []
test_x_init2 = []
test_n_init2 = []
test_y_init2 = []

for i in range(len(train_x_init)):
    for j in range(len(train_x_init[i])):
        train_x_init2.append(train_x_init[i][j])
        train_n_init2.append(train_n_init[i][j])
        train_y_init2.append(train_y_init[i][j])
        
for i in range(len(dev_x_init)):
    for j in range(len(dev_x_init[i])):
        dev_x_init2.append(dev_x_init[i][j])
        dev_n_init2.append(dev_n_init[i][j])
        dev_y_init2.append(dev_y_init[i][j])
        
for i in range(len(test_x_init)):
    for j in range(len(test_x_init[i])):
        test_x_init2.append(test_x_init[i][j])
        test_n_init2.append(test_n_init[i][j])
        test_y_init2.append(test_y_init[i][j])



# NEURAL BAG OF WORDS MODEL #

In [201]:
for i in range(10):

    print(train_x_init2[i])
    print(test_x_init2[i])
    print(dev_x_init2[i])


[2763, 169, 3496, 5259, 2937, 5259, 2157, 169, 5778, 873, 2350, 5453, 4366, 3630, 2183, 847, 2995, 3603, 4476, 5196, 4203, 169, 3603, 5778, 873, 1537, 5260, 169, 2350, 5453, 5102, 3630, 5557, 1653, 5259, 5201, 2608, 3425, 169, 2466, 169, 64, 668, 169, 4509, 169, 4509, 169, 2848, 169]
[3440, 246, 5259, 3378, 2407, 1067, 169, 2481, 4831, 5174, 5444, 1998, 204, 3235, 169, 169, 1608, 473, 169, 4509, 169, 4500, 3566, 12, 169, 4509, 169, 169, 4081]
[3440, 246, 5259, 3378, 2407, 1067, 169, 2481, 4831, 5174, 5444, 1998, 204, 3235, 169, 169, 1608, 473, 169, 4509, 169, 4500, 3566, 12, 169, 4509, 169, 169, 4081]
[2830, 1689, 882, 2756, 558, 169, 169, 2859, 169, 2787, 169, 2876, 169, 2788, 169, 5183, 169, 2869, 169, 3069, 169, 5723, 169, 1061, 169, 2174, 169, 5551, 169, 882, 169, 1689, 169, 2849, 169, 4509, 169, 4835, 169, 1690, 169, 2112, 3810]
[4508, 294, 169, 169, 169, 169, 2046, 4503, 1176, 169, 1234, 169, 169, 1347, 3566, 2148, 897, 12, 2971, 169, 2927, 169, 1160, 12, 871, 169, 4891, 12, 882,

In [202]:
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr ##I HAD TO REMOVE NS TO MAKE IT WORK< PLEASE TAKE A LOOK IF IT HAS ANY IMPACT

In [203]:
train_x = pad_np_array2(train_x_init2, max_len = 50, pad_id=0)
train_n = np.array(train_n_init2)
train_y = np.array(train_y_init2)

test_x = pad_np_array2(test_x_init2, max_len = 50, pad_id=0)
test_n = np.array(test_n_init2)
test_y = np.array(test_y_init2)

dev_x = pad_np_array2(dev_x_init2, max_len = 50, pad_id=0)
dev_n = np.array(dev_n_init2)
dev_y = np.array(dev_y_init2)

In [204]:
print(len(train_x))
print(len(train_n))
print(len(train_y))
print(len(dev_x))
print(len(dev_n))
print(len(dev_y))
print(len(test_x))
print(len(test_n))
print(len(test_y))

214320
214320
214320
16372
16372
16372
16608
16608
16608


In [205]:
lbl, cnts = np.unique(test_y, return_counts=True)
print(list(zip(lbl, cnts)))
print(lbl)
print(cnts)

# labels = 
# counts = 

plt.hist((cnts[0]*[lbl[0]],cnts[1]*[lbl[1]]))

[(0, 16608)]
[0]
[16608]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
lbl2, cnts2 = np.unique(train_y, return_counts=True)
print(list(zip(lbl2, cnts2)))
print(lbl2[0])
print(cnts2[0])

# labels = 
# counts = 

plt.hist((cnts2[0]*[lbl2[0]],cnts2[1]*[lbl2[1]]))


In [206]:
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 125

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 125 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

15,625 examples, moving-average loss 0.20
31,250 examples, moving-average loss 0.13
46,875 examples, moving-average loss 0.09
62,500 examples, moving-average loss 0.07
78,125 examples, moving-average loss 0.08
93,750 examples, moving-average loss 0.07
109,375 examples, moving-average loss 0.06
125,000 examples, moving-average loss 0.08
140,625 examples, moving-average loss 0.07
156,250 examples, moving-average loss 0.07
171,875 examples, moving-average loss 0.06
187,500 examples, moving-average loss 0.06
203,125 examples, moving-average loss 0.05
Completed one epoch in 0:00:10


In [207]:
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (5,855 words) written to '/tmp/tf_bow_sst_20181203-1844/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20181203-1844/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181203-1844', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c7718e630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181203-1844' --port 6006

Then in your browser, open: http:

In [208]:
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=125, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tf_bow_sst_20181203-1844/model.ckpt.
INFO:tensorflow:loss = 1.1960714, step = 1
INFO:tensorflow:global_step/sec: 131.023
INFO:tensorflow:loss = 0.28880808, step = 101 (0.766 sec)
INFO:tensorflow:global_step/sec: 141.037
INFO:tensorflow:loss = 0.17751586, step = 201 (0.709 sec)
INFO:tensorflow:global_step/sec: 140.911
INFO:tensorflow:loss = 0.11779082, step = 301 (0.709 sec)
INFO:tensorflow:global_step/sec: 143.973
INFO:tensorflow:loss = 0.17678559, step = 401 (0.695 sec)
INFO:tensorflow:global_step/sec: 140.34
INFO:tensorflow:loss = 0.85279834, step = 501 (0.712 sec)
INFO:tensorflow:global_step/sec: 140.174
INFO:tensorflow:loss = 0.095653996, step = 601 (0.714 sec)
INFO:tensorflow:global_step/sec: 140.0

INFO:tensorflow:global_step/sec: 142.049
INFO:tensorflow:loss = 0.05826583, step = 6731 (0.705 sec)
INFO:tensorflow:global_step/sec: 142.638
INFO:tensorflow:loss = 0.05732845, step = 6831 (0.702 sec)
INFO:tensorflow:Saving checkpoints for 6860 into /tmp/tf_bow_sst_20181203-1844/model.ckpt.
INFO:tensorflow:Loss for final step: 0.045052115.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-03-18:45:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181203-1844/model.ckpt-6860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-03-18:45:52
INFO:tensorflow:Saving dict for global step 6860: accuracy = 0.0, cross_entropy_loss = 4.9667664, global_step = 6860, loss = 5.014501
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6860: /tmp/tf_bow_sst_20181203-1844/model.ckpt-6860
INFO:tens

INFO:tensorflow:global_step/sec: 141.801
INFO:tensorflow:loss = 0.07804706, step = 12491 (0.706 sec)
INFO:tensorflow:global_step/sec: 141.957
INFO:tensorflow:loss = 0.057025738, step = 12591 (0.704 sec)
INFO:tensorflow:global_step/sec: 139.747
INFO:tensorflow:loss = 0.08086192, step = 12691 (0.715 sec)
INFO:tensorflow:global_step/sec: 143.205
INFO:tensorflow:loss = 0.07082445, step = 12791 (0.700 sec)
INFO:tensorflow:global_step/sec: 143.579
INFO:tensorflow:loss = 0.0683239, step = 12891 (0.696 sec)
INFO:tensorflow:global_step/sec: 144.048
INFO:tensorflow:loss = 0.06615277, step = 12991 (0.694 sec)
INFO:tensorflow:global_step/sec: 140.362
INFO:tensorflow:loss = 0.07297862, step = 13091 (0.713 sec)
INFO:tensorflow:global_step/sec: 142.965
INFO:tensorflow:loss = 0.06639216, step = 13191 (0.699 sec)
INFO:tensorflow:global_step/sec: 142.678
INFO:tensorflow:loss = 0.063797675, step = 13291 (0.702 sec)
INFO:tensorflow:global_step/sec: 141.732
INFO:tensorflow:loss = 0.07020141, step = 13391 (

In [209]:

# test_input_fn = None  # replace with an input_fn, similar to dev_input_fn

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  # replace with result of model.evaluate(...)

#### END(YOUR CODE) ####
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-03-18:47:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181203-1844/model.ckpt-17150
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-03-18:47:16
INFO:tensorflow:Saving dict for global step 17150: accuracy = 1.0, cross_entropy_loss = 0.008333357, global_step = 17150, loss = 0.057105113
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 17150: /tmp/tf_bow_sst_20181203-1844/model.ckpt-17150
Accuracy on test set: 100.00%


{'accuracy': 1.0,
 'cross_entropy_loss': 0.008333357,
 'loss': 0.057105113,
 'global_step': 17150}

In [210]:
from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
acc = accuracy_score(y_pred, test_y)

print("Accuracy on test set: {:.02%}".format(acc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181203-1844/model.ckpt-17150
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Accuracy on test set: 100.00%


In [211]:
# predicted labels
print(y_pred[0:50])
print(len(y_pred))
print(len(test_y))
print(predictions[:2])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
16608
16608
[{'proba': array([0.99221593, 0.0077841 ], dtype=float32), 'max': 0}, {'proba': array([0.99614114, 0.0038589 ], dtype=float32), 'max': 0}]


In [212]:
from sklearn.metrics import average_precision_score
    

average_precision = average_precision_score(y_pred,test_y)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

Average precision score: nan


/home/felixtsui/anaconda3/lib/python3.6/site-packages/sklearn/metrics/ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
